In [1]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)

2023-02-21 21:29:04.542197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 21:29:04.719782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 21:29:04.719843: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 21:29:05.917078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [4]:
#Setup Variables as examplegen_playground
import os

# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [17]:
import json
from typing import Any, Dict, List

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.types.standard_artifacts import Examples

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    # User input dictionary with split-names and their resepctive uri to tfrecords
  tfrecords_dict={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }
  split_names=''
  counter=1

  # Destination directory for source
  destination_examples_artifact_uri = './examples/'

  # Create a folder in destination_examples_artifact_uri for every key name
  for key in tfrecords_dict:
        split_name_temp = "Split-"+key
        path = os.path.join(destination_examples_artifact_uri, split_name_temp)
        if not os.path.exists(path):
            os.mkdir(path)
        shutil.copy(tfrecords_dict[key], path)

  # Create split_names string
  for key in tfrecords_dict:
      if(counter==1):
          split_names+='["'+key+'","'
          counter=counter+1
      elif(counter==2):
          split_names+=key+'"]'
      else:
          split_names+=key+','
          counter=counter+1

  print(split_names)
  
  # CopyExampleGen execution process to replace Importer Node functionality
  examples_importer = tfx.dsl.Importer(
        source_uri=destination_examples_artifact_uri,
        artifact_type=tfx.types.standard_artifacts.Examples,
        # properties={"split_names": '["train","eva"]'}
        properties={"split_names": split_names}
        ).with_id('examples_importer')
  
  # Test downstream component
  statistics_gen = tfx.components.StatisticsGen(
     examples=examples_importer.outputs['result'])

  # Following three components will be included in the pipeline.
  components = [
    #   examples_importer,
      examples_importer,
      statistics_gen
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [18]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/copy_tfrecords/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/copy_tfrecords/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component examples_importer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.importer.Importer"
  }
  id: "examples_importer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "copy_tfrecords"
      }
    }
  }
  contexts {
    type {
      n

["train","eval"]


INFO:absl:Statistics for split train written to ./artifacts/StatisticsGen/statistics/52/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to ./artifacts/StatisticsGen/statistics/52/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 52 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'statistics': [Artifact(artifact: uri: "./artifacts/StatisticsGen/statistics/52"
, artifact_type: name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)]}) for execution 52
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component StatisticsGen is finished.


In [ ]:
context = InteractiveContext()

split_names='["train","eval"]'

# Importer Node
examples_importer = tfx.dsl.Importer(
    source_uri='./artifacts/CsvExampleGen/examples/5/',
    artifact_type=tfx.types.standard_artifacts.Examples,
    properties={"split_names":split_names}).with_id(
        'examples_importer')

context.run(examples_importer)

# print("Output (examples_importer.outputs['result'].get()[0].version): ", examples_importer.outputs['result'].get()[0])

statistics_gen = tfx.components.StatisticsGen(
    examples=examples_importer.outputs['result'])

context.run(statistics_gen)

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

# _data_filepath = os.path.join(DATA_ROOT, "data.csv")

# print(data_filepath)
example_gen = tfx.components.CsvExampleGen(input_base=DATA_ROOT)

context.run(example_gen)

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import apache_beam as beam

from absl import logging

# from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
# from base_example_gen_executor_v2.py import BaseExampleGenExecutorV2
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=False)
print(ds)



In [ ]:
###################################################################################
### [USE CASE 1]
### Tfrecords are already created but not in Examples Artifact form

### This component will:
### 1. Accept a dict with with {'split_name': './path/to/split_name/tfrecords.gz'}
### 2. Add them to a folder to follow Example Artifact directory structure
### 3. Import them with Importer node to register the external resource into MLMD
###################################################################################
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Create pipeline to run Importer node
def _create_pipeline(
  pipeline_name: str,
  pipeline_root: str,
  data_root: str,
  metadata_path: str
  ) -> tfx.dsl.Pipeline:

  context = InteractiveContext()
  ### Get external source data
  # Source directory of external Examples Artifacts
  source_examples_artifact_uri = '../artifacts/'

  # Destination directory for source
  destination_examples_artifact_uri = './artifacts/'

  # Get all files from source_examples_artifact_uri
  files = os.listdir(source_examples_artifact_uri)

  # Import source files to destination
  shutil.copytree(source_examples_artifact_uri, destination_examples_artifact_uri, dirs_exist_ok=True)


  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print(example_gen.outputs['examples'])

  # Importer Node
  examples_importer = tfx.dsl.Importer(
      source_uri=TFRECORDS_DATA_PATH,
      artifact_type=tfx.types.standard_artifacts.Examples).with_id(
          'examples_importer')

  context.run(examples_importer)
  print("IMPORTER_NODE: ", examples_importer.outputs)
  print("IMPORTER_NODE: ", examples_importer.outputs['result'].additional_properties.keys)
  print(list(examples_importer.outputs['result'].additional_properties.keys()))


  # Test if downstream component accepts output from Importer Node
  statistics_gen = tfx.components.StatisticsGen(
      examples=examples_importer.outputs['result'])


  # EXAMPLEGEN COMPONENT OUTPUT
  # Brings data into the pipeline.
  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print("EXAMPLE_GEN: ", example_gen.outputs['examples'])

  # print("tfx.components.CsvExampleGen.outputs['examples'] output: \n\n",
  #   "Artifact Type: ", example_gen.outputs['examples']._artifact_type, "\n"
  #   "Producer Component Id: ", example_gen.outputs['examples'].producer_component_id, "\n"
  #   "Output Key: ", example_gen.outputs['examples'].output_key, "\n"
  #   "Additional Properties: ", example_gen.outputs['examples'].additional_properties, "\n"
  #   "Additional Custom Properties: ", example_gen.outputs['examples'].additional_custom_properties, "\n"
  # )
  # print(list(example_gen.outputs['examples'].additional_properties.keys()))

  components = [
    # example_gen,
    examples_importer,
    statistics_gen
  ]

  return tfx.dsl.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
    components=components
)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

In [ ]:
import re

dict_output = {
	"name": "ValueError",
	"message": "No splits for examples artifact: Artifact(artifact: id: 1\ntype_id: 14\nuri: \"./artifacts/CsvExampleGen/examples/5/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mValueError\u001b[0m                                Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_3107170/3844383043.py\u001b[0m in \u001b[0;36m<cell line: 16>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     14\u001b[0m     examples=examples_importer.outputs['result'])\n\u001b[1;32m     15\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 16\u001b[0;31m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mstatistics_gen\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/notebook_utils.py\u001b[0m in \u001b[0;36mrun_if_ipython\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     29\u001b[0m       \u001b[0;31m# __IPYTHON__ variable is set by IPython, see\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     30\u001b[0m       \u001b[0;31m# https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/reference.html#embedding-ipython.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 31\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     32\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     33\u001b[0m       logging.warning(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, component, enable_cache, beam_pipeline_args)\u001b[0m\n\u001b[1;32m    162\u001b[0m         \u001b[0mtelemetry_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLABEL_TFX_RUNNER\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mrunner_label\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    163\u001b[0m     }):\n\u001b[0;32m--> 164\u001b[0;31m       \u001b[0mexecution_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlauncher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_id\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    165\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    166\u001b[0m     return execution_result.ExecutionResult(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/base_component_launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    204\u001b[0m       \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    205\u001b[0m       \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 206\u001b[0;31m       self._run_executor(execution_decision.execution_id,\n\u001b[0m\u001b[1;32m    207\u001b[0m                          \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    208\u001b[0m                          \u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/in_process_component_launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_id, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     71\u001b[0m     \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m     \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 73\u001b[0;31m     executor.Do(\n\u001b[0m\u001b[1;32m     74\u001b[0m         copy.deepcopy(input_dict), output_dict, copy.deepcopy(exec_properties))\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/components/statistics_gen/executor.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m    154\u001b[0m           (split, tfxio_factory(io_utils.all_files_pattern(uri))))\n\u001b[1;32m    155\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 156\u001b[0;31m       \u001b[0;32mraise\u001b[0m \u001b[0mValueError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'No splits for examples artifact: %s'\u001b[0m \u001b[0;34m%\u001b[0m \u001b[0mexamples\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    157\u001b[0m     \u001b[0;32mwith\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_make_beam_pipeline\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0mp\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    158\u001b[0m       \u001b[0;32mfor\u001b[0m \u001b[0msplit\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtfxio\u001b[0m \u001b[0;32min\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mValueError\u001b[0m: No splits for examples artifact: Artifact(artifact: id: 1\ntype_id: 14\nuri: \"./artifacts/CsvExampleGen/examples/5/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)"
}

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)

In [ ]:
import re

dict_output = {
	"name": "JSONDecodeError",
	"message": "Expecting value: line 1 column 1 (char 0)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mJSONDecodeError\u001b[0m                           Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_3107170/380543175.py\u001b[0m in \u001b[0;36m<cell line: 17>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     15\u001b[0m     examples=examples_importer.outputs['result'])\n\u001b[1;32m     16\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 17\u001b[0;31m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mstatistics_gen\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/notebook_utils.py\u001b[0m in \u001b[0;36mrun_if_ipython\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     29\u001b[0m       \u001b[0;31m# __IPYTHON__ variable is set by IPython, see\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     30\u001b[0m       \u001b[0;31m# https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/reference.html#embedding-ipython.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 31\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     32\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     33\u001b[0m       logging.warning(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, component, enable_cache, beam_pipeline_args)\u001b[0m\n\u001b[1;32m    162\u001b[0m         \u001b[0mtelemetry_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLABEL_TFX_RUNNER\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mrunner_label\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    163\u001b[0m     }):\n\u001b[0;32m--> 164\u001b[0;31m       \u001b[0mexecution_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlauncher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_id\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    165\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    166\u001b[0m     return execution_result.ExecutionResult(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/base_component_launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    204\u001b[0m       \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    205\u001b[0m       \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 206\u001b[0;31m       self._run_executor(execution_decision.execution_id,\n\u001b[0m\u001b[1;32m    207\u001b[0m                          \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    208\u001b[0m                          \u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/in_process_component_launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_id, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     71\u001b[0m     \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m     \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 73\u001b[0;31m     executor.Do(\n\u001b[0m\u001b[1;32m     74\u001b[0m         copy.deepcopy(input_dict), output_dict, copy.deepcopy(exec_properties))\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/components/statistics_gen/executor.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     88\u001b[0m     examples = artifact_utils.get_single_instance(\n\u001b[1;32m     89\u001b[0m         input_dict[standard_component_specs.EXAMPLES_KEY])\n\u001b[0;32m---> 90\u001b[0;31m     examples_split_names = artifact_utils.decode_split_names(\n\u001b[0m\u001b[1;32m     91\u001b[0m         examples.split_names)\n\u001b[1;32m     92\u001b[0m     split_names = [\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/types/artifact_utils.py\u001b[0m in \u001b[0;36mdecode_split_names\u001b[0;34m(split_names)\u001b[0m\n\u001b[1;32m    209\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0msplit_names\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    210\u001b[0m     \u001b[0;32mreturn\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 211\u001b[0;31m   \u001b[0;32mreturn\u001b[0m \u001b[0mjson\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mloads\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0msplit_names\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    212\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    213\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/__init__.py\u001b[0m in \u001b[0;36mloads\u001b[0;34m(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)\u001b[0m\n\u001b[1;32m    355\u001b[0m             \u001b[0mparse_int\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m \u001b[0;32mand\u001b[0m \u001b[0mparse_float\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m \u001b[0;32mand\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    356\u001b[0m             parse_constant is None and object_pairs_hook is None and not kw):\n\u001b[0;32m--> 357\u001b[0;31m         \u001b[0;32mreturn\u001b[0m \u001b[0m_default_decoder\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdecode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    358\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0mcls\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    359\u001b[0m         \u001b[0mcls\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mJSONDecoder\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/decoder.py\u001b[0m in \u001b[0;36mdecode\u001b[0;34m(self, s, _w)\u001b[0m\n\u001b[1;32m    335\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    336\u001b[0m         \"\"\"\n\u001b[0;32m--> 337\u001b[0;31m         \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mraw_decode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0midx\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0m_w\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mend\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    338\u001b[0m         \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0m_w\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mend\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    339\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m!=\u001b[0m \u001b[0mlen\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/decoder.py\u001b[0m in \u001b[0;36mraw_decode\u001b[0;34m(self, s, idx)\u001b[0m\n\u001b[1;32m    353\u001b[0m             \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mscan_once\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0midx\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    354\u001b[0m         \u001b[0;32mexcept\u001b[0m \u001b[0mStopIteration\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0merr\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 355\u001b[0;31m             \u001b[0;32mraise\u001b[0m \u001b[0mJSONDecodeError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"Expecting value\"\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0merr\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mvalue\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mfrom\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    356\u001b[0m         \u001b[0;32mreturn\u001b[0m \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mJSONDecodeError\u001b[0m: Expecting value: line 1 column 1 (char 0)"
}

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)